In [1]:
from importlib import reload

import GPRutils
import vK2KGPR
import plotGPR

import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import astropy.stats as stats

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
dataC = GPRutils.dataContainer()
dataC.load()
dataC.splitData(nSigma=4, train_size=0.80)

Exposure: 364215
Band: z
Number of objects: 110646


INFO: Query finished. [astroquery.utils.tap.core]
There were 10847 matches within 0.5 arcsec.
110359 total detections
8446 training set detections
2112 validation set detections
99801 prediiction set detections


In [3]:
GP = vK2KGPR.vonKarman2KernelGPR(dataC, printing=True, outDir=".")
# GP.fitCorr()
# GP.optimize()
params = np.array([
    6.0052269,
    4.1389713,
    0.0271109,
    0.0624530,
    -0.0014108
])

In [4]:
GP.fit(params)

In [6]:
GP.predict(GP.dC.Xvalid)

In [ ]:
# dataC.quickPlot(savePath="preclipping")

In [ ]:
mask = stats.sigma_clip(
    dataC.Yvalid - dataC.fbar_s,
    sigma=4, axis=0).mask
mask = ~np.logical_or(*mask.T)

dataC.Xvalid = dataC.Xvalid[mask]
dataC.Yvalid = dataC.Yvalid[mask]
dataC.Evalid = dataC.Evalid[mask]

In [ ]:
GP.predict(GP.dC.Xtrain)

mask = stats.sigma_clip(
    dataC.Ytrain - dataC.fbar_s,
    sigma=4, axis=0).mask
mask = ~np.logical_or(*mask.T)

dataC.Xtrain = dataC.Xtrain[mask]
dataC.Ytrain = dataC.Ytrain[mask]
dataC.Etrain =  dataC.Etrain[mask]

In [ ]:
# To do this second round of optimization we will need to redfine nData,
# nTrain, and, nValid. Maybe other things too?

In [ ]:
# GP.optimize(v0=params)
params2 = np.array([
    2.22243044e+01,
    1.89448632e+00,
    -4.56342809e-04,
    9.20082711e-02,
    1.85203931e-03
])

In [ ]:
GP.fit(params2)

In [ ]:
GP.predict(GP.dC.Xvalid)

In [ ]:
# dataC.quickPlot(savePath="postclipping")